<a href="https://colab.research.google.com/github/Mario-RJunior/calculadora-imoveis/blob/master/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Machine Learning

Agora é a etapa em que iremos gerar um modelo usando técnicas de Machine Learning para gerar um modelo que irá nos ajudar a prever o valor de aluguéis de imóveis da cidade de São Paulo. Para isso usaremos como base os arquivos csv gerados anteriormente.